In [1]:
import math
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import time

## Read in the data

In [2]:
df = pd.read_csv('cardata_cleaned.csv', names=['buying','maint','doors','persons','lug_boot','safety','class'], skiprows =[0])
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,3,3,2,2,0,0,0
1,3,3,2,2,0,1,0
2,3,3,2,2,0,2,0
3,3,3,2,2,1,0,0
4,3,3,2,2,1,1,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
buying      1728 non-null int64
maint       1728 non-null int64
doors       1728 non-null int64
persons     1728 non-null int64
lug_boot    1728 non-null int64
safety      1728 non-null int64
class       1728 non-null int64
dtypes: int64(7)
memory usage: 94.6 KB


## split data into 70% training set and 30% test set

In [4]:
X, y = train_test_split(df, test_size = 0.3)

## Train the Decision Tree

In [5]:
# Calculate information entropy of an array in the df 
def expectedInfo(in_arr, b = 4):
    ent = 0
    ct = list(Counter(in_arr).values())
    prob = [x/sum(ct) for x in ct]
    for p in prob:
        if p != 0:
            ent += (-p * math.log(p, b))
    return ent

# Calculate information gain of attribute in df
def infoGain(data, attr, init_exp_info, cl_col = 'class', cl = [0, 1, 2, 3]):
    
    cl_count = {}
    inst = data.shape[0]
    g = init_exp_info
    b = len(cl)
    
    for i, r in data.iterrows():
        idx = cl.index(r[cl_col])
        attr_val = r[attr]
        if attr_val not in cl_count:
            cl_count[attr_val] = [1 if i == idx else 0 for i in range(b)]
        else:
            cl_count[attr_val][idx] += 1       
    
    for k, val in cl_count.items():
        inst_left = sum(val)
        prob = [x/inst_left for x in val]
        ent = 0
        for p in prob:
            if p != 0:
                ent -= p * math.log(p, b)
        g -= (inst_left/inst) * ent
    return g

# Store decision tree in nonbinary tree
class Node:
    def __init__(self, nm = 'root', dt = None, isLeaf = False):
        self.nm = nm
        self.dt = dt
        self.offspring = {}
        self.isLeaf = isLeaf
    def __str__(self):
        return str(self.nm)
    
    def findSib(self, tree_path):
        if tree_path[0] not in self.offspring:
            print('find_child: node ' + str(self.dt) + ' has no child with path: ' + tree_path[0])
        elif tree_path[1:]:
            return self.offspring[tree_path[0]].findSib(tree_path[1:])
        else:
            return self.offspring[tree_path[0]]
        
    def selectSib(self, tree_path, node):
        if tree_path[0] not in self.offspring:
            if tree_path[1:]:
                print('select_sibling: node ' + self.dt + ' has no child with path: ' + tree_path[0])
            else:
                self.offspring[tree_path[0]] = node
        else:
            self.offspring[tree_path[0]].selectSib(tree_path[1:], node)

# Print the decision tree
def printDecisionTree(node, lvl = 0, dt_path = ''):
    if lvl == 0:
        print('DTree: ' + str(node))
    else:
        print ('\t' * lvl + '└' + '─' + dt_path + '─' + str(node))
    if node.offspring:
        for k in node.offspring:
            printDecisionTree(node.offspring[k], lvl + 1, str(k))
            
# Construct the decision tree using recursion
def constructDecisionTree(node, left_attr, cl_col = 'class', cl = [0, 1, 2, 3]):
    
    # Initialize the tree if the node is root
    if node.nm == 'root':
        cur_ent = expectedInfo(list(node.dt[cl_col]))
        max_g = 0
        max_g_attr = ''
        for attr in left_attr:
            gs = 0
            gs = infoGain(node.dt, attr, cur_ent)
            if gs >= max_g:
                max_g = gs
                max_g_attr = attr
        
        left_attr.remove(max_g_attr)
        child_node = Node(max_g_attr, node.dt)
        node.selectSib([' '], child_node)
        constructDecisionTree(node.findSib([' ']), list(left_attr))
    
    # If node is not root, build the tree
    else:
        attr = node.nm
        attr_vals = set(node.dt[attr])
        prev_df = node.dt
        
        for attr_val in attr_vals:
            cur_df = prev_df.loc[prev_df[attr] == attr_val]
            cur_ent = expectedInfo(list(cur_df[cl_col]))
            if cur_ent != 0 and left_attr:
                max_g = 0
                max_g_attr = ''
                for l_attr in left_attr:
                    gs = infoGain(cur_df, l_attr, cur_ent)
                    if gs >= max_g:
                        max_g = gs
                        max_g_attr = l_attr
                lf = list(left_attr)
                lf.remove(max_g_attr)
                child_node = Node(max_g_attr, cur_df)
                node.selectSib([attr_val], child_node)
                constructDecisionTree(node.findSib([attr_val]), list(lf))
                
            else:
                classify = Counter(cur_df[cl_col]).most_common(1)[0][0]
                child_node = Node(classify, isLeaf = True)
                node.selectSib([attr_val], child_node)
            

info gain "buying"	 =  0.04822448458480691
info gain "maint"	 =  0.0368519734607429
info gain "doors"	 =  0.002242858313316054
info gain "persons"	 =  0.1098314816699541
info gain "lug_boot"	 =  0.015004070623802712
info gain "safety"	 =  0.13109217827713193


In [12]:
names=['buying','maint','doors','persons','lug_boot','safety']
carEvalTree = Node('root', X)

start_time = time.time()
constructDecisionTree(carEvalTree, names)
print("--- %s seconds ---" % (time.time() - start_time))
print("")

printDecisionTree(carEvalTree)

--- 1.2632768154144287 seconds ---

DTree: root
	└─ ─safety
		└─0─0
		└─1─persons
			└─2─0
			└─4─buying
				└─0─maint
					└─0─lug_boot
						└─0─1
						└─1─doors
							└─3─1
							└─5─2
						└─2─2
					└─1─lug_boot
						└─0─1
						└─1─doors
							└─2─1
							└─3─1
							└─5─2
						└─2─2
					└─2─1
					└─3─lug_boot
						└─0─0
						└─1─doors
							└─2─0
							└─3─0
							└─5─1
						└─2─1
				└─1─maint
					└─0─lug_boot
						└─0─1
						└─1─doors
							└─2─1
							└─3─1
							└─4─2
							└─5─2
						└─2─2
					└─1─1
					└─2─lug_boot
						└─0─0
						└─1─doors
							└─2─0
							└─4─1
							└─5─1
						└─2─1
					└─3─lug_boot
						└─0─0
						└─1─doors
							└─2─0
							└─3─0
							└─4─1
							└─5─1
						└─2─1
				└─2─lug_boot
					└─0─0
					└─1─doors
						└─2─0
						└─3─0
						└─4─maint
							└─1─1
							└─2─1
							└─3─0
						└─5─maint
							└─0─1
							└─1─1
							└─2─1
							└─3─0
					└─2─maint
						└─0─1
						└─1─1
						└─2─1
						└─3